<a href="https://colab.research.google.com/github/bnnguyen/DESLab_ML_training_2024/blob/main/Deslab_2024_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classification using Wine data (UCI ML Repo)

Kỹ thuật [Naive Bayes Classifier](http://dataaspirant.com/2017/02/06/naive-bayes-classifier-machine-learning/) dựa trên cái gọi là định lý Bayesian và đặc biệt phù hợp khi tính chiều của đầu vào cao. Mặc dù đơn giản nhưng Naive Bayes thường có thể hoạt động tốt hơn các phương pháp phân loại phức tạp hơn.

### Định lý Bayes

Thuật toán này dựa trên [định lý __Bayes__](https://en.wikipedia.org/wiki/Bayes%27_theorem) nổi tiếng được đặt theo tên của Mục sư Thomas Bayes. Nó hoạt động dựa trên xác suất có điều kiện. [Xác suất có điều kiện](https://en.wikipedia.org/wiki/Conditional_probability) là xác suất để một điều gì đó sẽ xảy ra, vì một điều gì đó khác đã xảy ra. Sử dụng xác suất có điều kiện, chúng ta có thể tính xác suất của một sự kiện bằng kiến thức trước đó của nó.

Định lý Bayes được phát biểu về mặt toán học như phương trình sau:

$${\displaystyle P(A\mid B)={\frac {P(B\mid A)\,P(A)}{P(B)}},}$$
trong đó $A$ và $B$ là các sự kiện và $P(B)\neq{0}$.

$P(A\mid B)$ là [xác suất có điều kiện](https://en.wikipedia.org/wiki/Conditional_probability): khả năng xảy ra sự kiện $A$ nếu $B$ là đúng.

$P(B\mid A)$ cũng là một xác suất có điều kiện: khả năng xảy ra sự kiện $B$ nếu $A$ là đúng.

$P(A)$ và $P(B)$ là xác suất quan sát $A$ và $B$ độc lập với nhau; đây được gọi là [xác suất cận biên](https://en.wikipedia.org/wiki/Marginal_probability).

### _Naive_ trong Naive Bayes là gì và tại sao nó lại là thuật toán siêu nhanh?

Thuật toán được gọi với cái tên là Bayes ngây thơ hay Bayes ngốc nghếch vì việc tính toán xác suất cho mỗi giả thuyết được đơn giản hóa để làm cho việc tính toán của chúng trở nên dễ thực hiện. Thay vì cố gắng tính toán các giá trị của từng giá trị thuộc tính, chúng được giả định là độc lập có điều kiện với giá trị đích.

Đây là một giả định chắc chắn khó có thể xảy ra trong dữ liệu thực, tức là các thuộc tính không tương tác. Tuy nhiên, cách tiếp cận này thực hiện tốt một cách đáng ngạc nhiên đối với dữ liệu mà giả định này không đúng.

Việc huấn luyện diễn ra nhanh vì chỉ cần tính xác suất của từng lớp và xác suất của mỗi lớp với các giá trị đầu vào khác nhau. **Không cần phải điều chỉnh hệ số bằng quy trình tối ưu hóa.**

Xác suất của lớp chỉ đơn giản là tần số của các trường hợp thuộc về mỗi lớp chia cho tổng số trường hợp. Xác suất có điều kiện là tần số của từng giá trị thuộc tính đối với một giá trị lớp nhất định chia cho tần suất của các thể hiện có giá trị lớp đó.

### Dữ liệu được phân tích trong sổ ghi chép này

Sau đây chúng ta sẽ tìm hiểu cách sử dụng phương pháp Naive Bayes của Python scikit-learn để phân loại nguồn gốc rượu vang dựa trên dữ liệu phân tích hóa lý. Những dữ liệu này là kết quả phân tích hóa học về rượu vang được trồng trong cùng một vùng ở Ý nhưng có nguồn gốc từ ba giống cây trồng khác nhau. Phân tích đã xác định số lượng của 13 thành phần được tìm thấy trong mỗi loại rượu trong số ba loại rượu vang.

Thông tin chi tiết có thể được [**tìm thấy ở đây**](http://archive.ics.uci.edu/ml/datasets/Wine).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Data set

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/arjun-prabhakar/crimson-data-science/main/wine.data.csv')
df.head(10)


#### Basic statistics of the features

In [ ]:
df.iloc[:,1:].describe()

#### Boxplots by output labels/classes

In [ ]:
for c in df.columns[1:]:
    df.boxplot(c,by='Class',figsize=(7,4),fontsize=14)
    plt.title("{}\n".format(c),fontsize=16)
    plt.xlabel("Wine Class", fontsize=16)

**Có thể thấy rằng một số tính năng phân loại nhãn rượu vang khá rõ ràng.** Ví dụ: Alcalinity, Total Phenols hoặc Flavonoid tạo ra các ô hình hộp có các đường trung bình được phân tách rõ ràng, biểu thị rõ ràng các loại rượu vang.

Dưới đây là một ví dụ về phân tách lớp bằng hai biến

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['OD280/OD315 of diluted wines'],df['Flavanoids'],c=df['Class'],edgecolors='k',alpha=0.8,s=100)
plt.grid(True)
plt.title("Scatter plot of two features showing the \ncorrelation and class seperation",fontsize=15)
plt.xlabel("OD280/OD315 of diluted wines",fontsize=15)
plt.ylabel("Flavanoids",fontsize=15)

#### Các tính năng có độc lập không? Vẽ ma trận hiệp phương sai

Có thể thấy rằng có một số mối tương quan tốt giữa các đặc điểm, tức là chúng không độc lập với nhau, như giả định trong kỹ thuật Naive Bayes. Tuy nhiên, chúng tôi vẫn sẽ tiếp tục và áp dụng trình phân loại để xem hiệu suất của nó.

In [ ]:
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure(figsize=(16,12))
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Wine data set features correlation\n',fontsize=15)
    labels=df.columns
    ax1.set_xticklabels(labels,fontsize=9)
    ax1.set_yticklabels(labels,fontsize=9)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[0.1*i for i in range(-11,11)])
    plt.show()

correlation_matrix(df)

## Naive Bayes Classification

#### Test/train split

In [ ]:
from sklearn.model_selection import train_test_split

 # Test-set 30%
test_size = 0.3

In [ ]:
#Isolate the features from the Target variable:
# Meaning separate Class from the rest of the columns
# Y should contain only Class while X should contain the rest of the columns in the original dataset
X = df.drop('Class', axis=1)
y = df['Class']
# Specify the train and test X and Y using the imported train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [ ]:
X_train.shape

In [ ]:
#Look at head of X train
X_train.head()

#### Classification using GaussianNB

Cho một biến lớp $y$ và một vectơ đặc trưng phụ thuộc $x_1$ đến $x_n$, định lý Bayes phát biểu mối quan hệ sau:

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots x_n \mid y)} {P(x_1, \dots, x_n)}$$
Sử dụng giả định độc lập ngây thơ rằng
$$P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y),$$
với mọi $i$, mối quan hệ này được đơn giản hóa thành
$$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)} {P(x_1, \dots, x_n )}$$

Vì $P(x_1, \dots, x_n)$ là hằng số với đầu vào nên chúng ta có thể sử dụng quy tắc phân loại sau:
$$P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)$$
$$\Downarrow$$
$$\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),$$

và chúng ta có thể sử dụng ước tính [**Maximum A Posteriori**](https://en.wikipedia.org/wiki/Maximum_a_postteriori_estimation) (MAP) để ước tính $P(y)$ và $P(x_i \mid y)$ ; cái trước là tần số tương đối của lớp $y$ trong tập huấn luyện.

***GaussianNB ()*** triển khai thuật toán Gaussian Naive Bayes để phân loại. **Khả năng của các tính năng được giả định là Gaussian**:

$$ P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y }) $$

Các tham số $\sigma_y$ và $\mu_y$ được ước tính bằng khả năng tối đa.

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nbc = GaussianNB()

In [ ]:
nbc.fit(X_train, y_train)

#### Prediction và confusion matrix

In [ ]:
# Predict the class on the X_test
y_pred = nbc.predict(X_test)
#Find the total number of mislabeled data points
mislabel = np.sum(y_test!=y_pred)
print("Total number of mislabeled data points from {} test samples is {}".format(len(y_test), mislabel))

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = (confusion_matrix(y_test,y_pred))
cmdf = pd.DataFrame(cm,index=['Class 1','Class 2',' Class 3'], columns=['Class 1','Class 2',' Class 3'])
print("The confusion matrix looks like following...\n")
cmdf

**Điều này cho thấy rằng ngay cả khi có sự tương thích giữa các đặc điểm, thuật toán Naive Bayes vẫn hoạt động khá tốt và có thể tách các lớp một cách dễ dàng**